In [1]:
# Install packages if not available in Jupyter Notebook
# !pip install pandas
# !pip install ipywidgets
# !pip install pycountry_convert
# !pip install plotly
# !pip install matplotlib
# !pip install openpyxl
# !pip install IPython

In [2]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import pycountry_convert as pc
import plotly.graph_objs as go

In [3]:
# import the data
cv_df = pd.read_excel("Covid Protection Efficacy By Country.xlsx")

## This is the dataset of the Covid Efficacy and here the data will be cleaned such as dropping any rows with no values because no information can be found on the country thus is useless

In [4]:
cv_df.head()

,COUNTRY,ISO3,WHO_REGION,DATE_UPDATED,VACCINES_USED,NUMBER_VACCINES_TYPES_USED,"Protection against ORIGINAL Var- SEVERE, % average","Protection against ORIGINAL Var- INFECTION, % average","Protection against LATEST var BA1.n SEVERE, % average","Protection against LATEST var BA1.n INFECTION, % average",...,% Population vaccinated,% People Protected for ORGINAL SEVERE,% People Protected for ORGINAL INFECTION,% People Protected for OMICRON SEVERE,% People Protected for OMICRON INFECTION,% SUCEPTIBLE for BREAKTHROUGH ORGINAL SEVERE,% SUCEPTIBLE for BREAKTHROUGH ORIGINAL INFECTION,% SUCEPTIBLE for BREAKTHROUGH OMICRON SEVERE,% SUCEPTIBLE for BREAKTHROUGH OMICRON INFECTION,% Population NOT VACCINATED
0,Afghanistan,AFG,EMRO,2022-12-22,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",11.0,83.0,73.0,61.0,38.0,...,28.41,23.5803,20.7393,17.3301,10.7958,4.8297,7.6707,11.0799,17.6142,71.59
1,Albania,ALB,EURO,2022-12-11,"AstraZeneca - Vaxzevria,Gamaleya - Gam-Covid-V...",5.0,85.0,69.0,64.0,37.0,...,47.31,40.2135,32.6439,30.2784,17.5047,7.0965,14.6661,17.0316,29.8053,52.69
2,Algeria,DZA,AFRO,2022-09-04,"Beijing CNBG - BBIBP-CorV,Gamaleya - Gam-Covid...",4.0,77.0,66.0,57.0,35.0,...,17.46,13.4442,11.5236,9.9522,6.1110,4.0158,5.9364,7.5078,11.3490,82.54
3,American Samoa,ASM,WPRO,2022-08-23,"Janssen - Ad26.COV 2-S,Moderna - Spikevax,Pfiz...",3.0,93.0,83.0,67.0,42.0,...,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,100.00
4,Andorra,AND,EURO,2022-11-13,"AstraZeneca - Vaxzevria,Moderna - Spikevax,Pfi...",3.0,95.0,80.0,72.0,43.0,...,72.52,68.8940,58.0160,52.2144,31.1836,3.6260,14.5040,20.3056,41.3364,27.48


In [5]:
# Null values are dropped
# cv_df.isnull().sum()
cv_df = cv_df.dropna()
# cv_df.isnull().sum()

In [6]:
# Get a list of all unique vaccines used in the world
all_vaccines = cv_df['VACCINES_USED'].str.split(',').tolist()
l = []
for i in all_vaccines:
    for j in i:
        if '-' in j:
            l.append(j.split('-')[0])
        else:
            l.append(j.split()[0])
vaccines_list = list(set([vaccine.strip() for vaccine in l]))
vaccines_list.sort()

# A World Map with the distribution of vaccines around the world based on vaccine types

In [7]:
# do a world map to find distribution of vaccines around the world based on vaccine types
def chlo(vaccine):
    vaccine_df = cv_df[cv_df['VACCINES_USED'].str.contains(vaccine)] 
    fig1 = px.choropleth(vaccine_df,
                        locations='ISO3',
                        hover_name='COUNTRY',
                        hover_data={'COUNTRY': True, 'NUMBER_VACCINES_TYPES_USED': True,},
                        labels={'VACCINES_USED': 'Vaccines Used'},
                        title=f"Countries using {vaccine}"
                       )
    fig1.show()

# Create a dropdown widget from the vaccine list
vaccine_dropdown = widgets.Dropdown(options=vaccines_list,
                                    description='Vaccine:', 
                                    layout={'width': '300px'})

# Attach the chlo function to the dropdown widget
widgets.interact(chlo, vaccine=vaccine_dropdown)

interactive(children=(Dropdown(description='Vaccine:', layout=Layout(width='300px'), options=('Anhui ZL', 'Ast…

<function __main__.chlo(vaccine)>

# Bar Chart of List of Vaccines Distributed Around the World

In [8]:
# store the vaccine counts for unique vaccines around the world
vaccine_counts = {}
for vaccine in vaccines_list:
    vaccine_counts[vaccine] = 0

# Loop through the rows of the dataframe
for vaccines in all_vaccines:
    # Loop through the vaccines used by each country
    for vaccine in vaccines:
        # Loop through each vaccine in the vaccines_list 
        for vac in vaccines_list:
            # check if it is in the list of vaccines used by the country
            if vac in vaccine:
                # if the vaccine is in the list, then increment the count and store it in dictionary
                vaccine_counts[vac] += 1

name = list(vaccine_counts.keys())
number = list(vaccine_counts.values())

# bar chart for the dictionary of vaccines and the number of the vaccines
dat_vac = go.Bar(
    x = name,
    y = number,
    name = "Vaccine Distribution Count"
)
layout_vac = go.Layout(title = 'Vaccination Distribution', legend_title='Vaccine Lists', xaxis_title='Country', yaxis_title='Number of Vaccines')
fig_vac = go.Figure(data = dat_vac, layout = layout_vac)

fig_vac.show()

# Scatter Plot of the information of the Columns of Excel Sheet Such as % Vaccinated, % Protected from virus, etc when you hover

In [9]:
def sca6(col):
    fig1 = px.scatter_geo(cv_df,
                        locations='ISO3',
                        color='WHO_REGION',
                        hover_name='COUNTRY',
                        hover_data=[col],
    )
    display(fig1)
SELECT_COLUMN = widgets.Dropdown(options=list(cv_df.columns[5:22]), value=cv_df.columns[5], 
                                 description='Columns:', layout={'width': '300px'})

# Attach the scatter_geo plot function to the dropdown widget
widgets.interact(sca6, col=SELECT_COLUMN)

interactive(children=(Dropdown(description='Columns:', layout=Layout(width='300px'), options=('NUMBER_VACCINES…

<function __main__.sca6(col)>

# Countries in WHO_REGION are Less so Decided to Compare based on Continent

In [10]:
# I was planning to use who_region to find more information on people vaccinated, etc however there are less countries on some region
# and only 1 on other so I decided to use continents to find more information
who_region = cv_df.groupby('WHO_REGION')['COUNTRY'].nunique()
print(who_region)

WHO_REGION
AFRO     47
AMRO     48
EMRO     22
EURO     60
OTHER     1
SEARO    10
WPRO     29
Name: COUNTRY, dtype: int64


In [11]:
# To add column on continents I decided to use py_country to find continents based on country iso3 code however the way some countries are
# named does not seem to work so I decided to correct the information on the countries on specific index
row_updates = {24: "Bolivia", 49: "Ivory Coast", 52: 'Curaçao', 97: 'Iran', 154: 'Palestine', 168: 'South Korea', 204: 'United Kingdom', 223: 'Venezuela'}
for i, country in row_updates.items():
    cv_df.loc[i, 'COUNTRY'] = country

In [12]:
# copy the data information because I will be changing the countries
df_new = cv_df.copy()

In [13]:
# df_new.head()

In [14]:
# Define a function to get the continent based on ISO3 code
def convert(country):
    code = pc.country_name_to_country_alpha2(country.COUNTRY, cn_name_format = 'default')
    conti = pc.country_alpha2_to_continent_code(code)
    return conti
# Remove the continents that cannot be found on ISO3 code 
count_remove = ['Kosovo', 'Pitcairn Islands', 'Sint Maarten', 'Timor-Leste']
for country in count_remove:
    df_new = df_new[df_new['COUNTRY'] != country]
# Create a new column and apply the continents in that column
df_new['Continent'] = df_new.apply(convert, axis=1)


In [15]:
# df_new.head()

In [16]:
# since they are abbreviated change it to the full continent name
conti = {"AS": "Asia", "SA": "South America", "NA": "North America", "OC": "Oceania", "EU": "Europe", "AF": "Africa"}
df_new["Continent"] = df_new["Continent"].map(conti)

# This is the Dataset that has the Continent Column for Better Comparison

In [17]:
df_new.head()

,COUNTRY,ISO3,WHO_REGION,DATE_UPDATED,VACCINES_USED,NUMBER_VACCINES_TYPES_USED,"Protection against ORIGINAL Var- SEVERE, % average","Protection against ORIGINAL Var- INFECTION, % average","Protection against LATEST var BA1.n SEVERE, % average","Protection against LATEST var BA1.n INFECTION, % average",...,% People Protected for ORGINAL SEVERE,% People Protected for ORGINAL INFECTION,% People Protected for OMICRON SEVERE,% People Protected for OMICRON INFECTION,% SUCEPTIBLE for BREAKTHROUGH ORGINAL SEVERE,% SUCEPTIBLE for BREAKTHROUGH ORIGINAL INFECTION,% SUCEPTIBLE for BREAKTHROUGH OMICRON SEVERE,% SUCEPTIBLE for BREAKTHROUGH OMICRON INFECTION,% Population NOT VACCINATED,Continent
0,Afghanistan,AFG,EMRO,2022-12-22,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",11.0,83.0,73.0,61.0,38.0,...,23.5803,20.7393,17.3301,10.7958,4.8297,7.6707,11.0799,17.6142,71.59,Asia
1,Albania,ALB,EURO,2022-12-11,"AstraZeneca - Vaxzevria,Gamaleya - Gam-Covid-V...",5.0,85.0,69.0,64.0,37.0,...,40.2135,32.6439,30.2784,17.5047,7.0965,14.6661,17.0316,29.8053,52.69,Europe
2,Algeria,DZA,AFRO,2022-09-04,"Beijing CNBG - BBIBP-CorV,Gamaleya - Gam-Covid...",4.0,77.0,66.0,57.0,35.0,...,13.4442,11.5236,9.9522,6.1110,4.0158,5.9364,7.5078,11.3490,82.54,Africa
4,Andorra,AND,EURO,2022-11-13,"AstraZeneca - Vaxzevria,Moderna - Spikevax,Pfi...",3.0,95.0,80.0,72.0,43.0,...,68.8940,58.0160,52.2144,31.1836,3.6260,14.5040,20.3056,41.3364,27.48,Europe
5,Angola,AGO,AFRO,2022-12-18,SII - Covishield,1.0,94.0,63.0,71.0,36.0,...,39.8278,26.6931,30.0827,15.2532,2.5422,15.6769,12.2873,27.1168,57.63,Africa


In [18]:
# The continents that I removed before apply them from old dataset to new dataset
count = {'Kosovo': 'Europe', 'Pitcairn Islands': 'Oceania', 'Sint Maarten': 'North America', 'Timor-Leste': 'Asia'}
for country, continents in count.items():
    row_to_copy = cv_df.loc[cv_df['COUNTRY'] == country]
    df_new = pd.concat([df_new, row_to_copy], ignore_index=True)
    df_new.loc[df_new['COUNTRY'] == country, 'Continent'] = continents

# World Map based on Number of Vaccines Used 

# This is where we will do comparison whether there is a correlation between number of vaccines and people being vaccinated

In [19]:
fig = px.choropleth(df_new,
                    locations='ISO3',
                    color='NUMBER_VACCINES_TYPES_USED',
                    hover_name='COUNTRY',
                    hover_data=['NUMBER_VACCINES_TYPES_USED'],
                    animation_frame='Continent'
)
fig.show()

In [20]:
# Find the continents that are in the column continent
all_continent = df_new['Continent']
continent_list = []
for i in all_continent:
    if i not in continent_list:
        continent_list.append(i)
countries = set(continent_list)
print(countries)

{'Asia', 'Europe', 'North America', 'Africa', 'South America', 'Oceania'}


# Number of Vaccines Used and % Vaccinated Scatter Plot
## Are more people vaccinated as there are more vaccines?

## We see that even with more vaccines around it does not mean that there is going to be more people vaccinated which we see especially in Africa that even if they have one vaccine, there are some countries that have more people vaccinated with one vaccine than a country that has more than 10. In some cases there are some that have more vaccines and more people are vaccinated compared to other countries. Such include some countries in Asia as there are more vaccines, more people are likely to be vaccinated but not all countries are like that.

In [21]:
# Using the continents find out the number of vaccines and the vaccines used and % vaccinated in each continent
data_num = {}
for i in countries:
    # find the country, number of vaccines and vaccines used in a continent along with population vaccinated
    af_num = df_new.loc[df_new['Continent']== i, ['COUNTRY', 'NUMBER_VACCINES_TYPES_USED', 'VACCINES_USED']]
    os_num = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% Population vaccinated']]
    # create a scatter plot of the vaccinations
    data_num[i] = go.Scatter(
            x=af_num['NUMBER_VACCINES_TYPES_USED'], 
            y=os_num['% Population vaccinated'], 
            name="Number of Vaccinations vs People Vaccinated", 
            mode='markers', 
            marker=dict(size=8), 
            text=af_num['COUNTRY'] + '<br>' + af_num['VACCINES_USED']
        )
layout_num = go.Layout(title='Number of Vaccinations vs People Vaccinated', legend_title='Vaccination Status', xaxis_title='Number of Vaccines', yaxis_title='% Vaccinated')

def cont_vac(countr_vac):
    fig_num = go.Figure(data=data_num[countr_vac], layout=layout_num)
    fig_num.show()

# create a widget of each continent to find the information
countr_vac1 = widgets.Dropdown(options=countries,
                                    description='Continents:', 
                                    layout={'width': '300px'})

# Attach the cont_vac function to the dropdown widget
widgets.interact(cont_vac, countr_vac=countr_vac1)
    

interactive(children=(Dropdown(description='Continents:', layout=Layout(width='300px'), options=('Asia', 'Euro…

<function __main__.cont_vac(countr_vac)>

In [22]:
# find number of countries in each continent
countries_by_continent = df_new.groupby('Continent')['COUNTRY'].nunique()
# for continent, num_countries in countries_by_continent.items():
#     print(f"{continent}: {num_countries}")

In [23]:
# number of countries in a continent multiplied by 100
counts = {}
# store number of percentage vaccinated 
vaccinated = {}
# store number of percentage not vaccinated 
non_vaccinated = {}

# store breakthrough original severe
breakthrough = {}
# store original severe
original = {}
# store breakthrough original infection
breakthrough1 = {}
# store original infection
original1 = {}
# store breakthrough omicron severe
breakthrough2 = {}
# store omicron severe
original2 = {}
# store breakthrough omicron infection
breakthrough3 = {}
# store omicron infection
original3 = {}   

# loop through the number of countries in a continent
for continent, num_countries in countries_by_continent.items():
    # multiply the number by 100
    counts[continent] = 100 * num_countries
    # sum the percentage of people vaccinated
    vaccinated[continent] = df_new[df_new['Continent'] == continent]['% Population vaccinated'].sum()
    # sum the percentage of people not vaccinated
    non_vaccinated[continent] = df_new[df_new['Continent'] == continent]['% Population NOT VACCINATED'].sum()
    # sum the percentage of people protected for original severe
    original[continent] = df_new[df_new['Continent'] == continent]['% People Protected for ORGINAL SEVERE'].sum()
    # sum the percentage of people susceptible of breakthrough original severe 
    breakthrough[continent] = df_new[df_new['Continent'] == continent]['% SUCEPTIBLE for  BREAKTHROUGH  ORGINAL SEVERE'].sum()
    # sum the percentage of people protected for original infection
    original1[continent] = df_new[df_new['Continent'] == continent]['% People Protected for ORGINAL INFECTION'].sum()
    # sum the percentage of peoplesusceptible of breakthrough original infection
    breakthrough1[continent] = df_new[df_new['Continent'] == continent]['% SUCEPTIBLE for  BREAKTHROUGH ORIGINAL  INFECTION'].sum()
    # sum the percentage of people protected for omicron severe
    original2[continent] = df_new[df_new['Continent'] == continent]['% People Protected for OMICRON SEVERE'].sum()
    # sum the percentage of people susceptible of breakthrough omicron severe 
    breakthrough2[continent] = df_new[df_new['Continent'] == continent]['% SUCEPTIBLE for  BREAKTHROUGH OMICRON SEVERE'].sum()
    # sum the percentage of people protected for omicron infection
    original3[continent] = df_new[df_new['Continent'] == continent]['% People Protected for OMICRON INFECTION'].sum()
    # sum the percentage of people susceptible of breakthrough omicron infection
    breakthrough3[continent] = df_new[df_new['Continent'] == continent]['% SUCEPTIBLE for  BREAKTHROUGH OMICRON INFECTION'].sum()

# store the values found after summing the percentage, divide it with the continents and the average percentage found store them in the variables
val = {continent: vaccinated[continent] / counts[continent] for continent in counts}
val2 = {continent: non_vaccinated[continent] / counts[continent] for continent in counts}
val3 = {continent: breakthrough[continent] / counts[continent] for continent in counts}
val4 = {continent: original[continent] / counts[continent] for continent in counts}
val5 = {continent: breakthrough1[continent] / counts[continent] for continent in counts}
val6 = {continent: original1[continent] / counts[continent] for continent in counts}
val7 = {continent: breakthrough2[continent] / counts[continent] for continent in counts}
val8 = {continent: original2[continent] / counts[continent] for continent in counts}
val9 = {continent: breakthrough3[continent] / counts[continent] for continent in counts}
val10 = {continent: original3[continent] / counts[continent] for continent in counts}

# Vaccination Status Based on Continent

## Since Number of Vaccines received does not matter I want to compare how many people were and weren't vaccinated based on continents to get more understanding on whether continent regions impact people being vaccinated. From there I want to compare how people in different contries and continents fare on handling the viruses and infections as more or less people are vaccinated and how likely are they to be protected against virus. 

In [24]:
# Since they are stored in dictionary, take the values and keys of the % vaccinated and % not vaccinated and use them to make 
# a stacked bar graph to show the comparison of the people vaccinated and not vaccinated as a whole
country = list(val.keys())
country2 = list(val2.keys())
vaccinated = list(val.values())
non_vaccinated = list(val2.values())
# first bar with people vaccinated
data01 = go.Bar(
    x = country,
    y = vaccinated,
    name = "% Vaccinated"
)
# second bar with people not vaccinated
data11 = go.Bar(
    x = country2,
    y = non_vaccinated,
    name = "% Not Vaccinated"
)

# store all of them together
data = [data01, data11]

# create the bar graph in stack
layout = go.Layout(title = 'Vaccination Status by Continent', barmode = 'stack', legend_title='Vaccination Status', xaxis_title='Continent', yaxis_title='% Percentage Vaccinated and Not Vaccinated')
fig = go.Figure(data = data, layout = layout)

fig.show()


# Bar Graph of Breakthrough Susceptible and Protection against Virus

## We see that all of the continents are more likely to be protected from the virus as there a higher percentage and even though they are vaccinated, it does not mean that they won't get the virus meaning that they will be susceptible for breakthrough virus. The good thing is that there are less percentage of people who are likely to get breakthrough virus and infection so it shows that virus does do a good job in protecting large percentage of people as they are higher percentage for protection against virus and infection compared to breakthrough which is not a lot of percentage.

In [25]:
# Since they are stored in dictionary, take the values and keys of the original virus and infection and 
# breakthrough virus and infection and use them to make 
# a bar graph to show the comparison of the people that are vaccinated how many of them would be protected and how many
# even if they are vaccinated would get the virus and the infections
country1 = list(val3.keys())
country21 = list(val4.keys())
breakthrough = list(val3.values())
original = list(val4.values())
country13 = list(val5.keys())
country213 = list(val6.keys())
breakthrough1 = list(val5.values())
original1 = list(val6.values())
# breakthrough bar graph
databr = go.Bar(
    x = country1,
    y = breakthrough,
    name = "Breakthrough"
)
# original virus bar graph
dataor = go.Bar(
    x = country21,
    y = original,
    name = "Original"
)
# breakthrough infection bar graph
databr1 = go.Bar(
    x = country13,
    y = breakthrough1,
    name = "Breakthrough Infection"
)
# original infection bar graph
dataor1 = go.Bar(
    x = country213,
    y = original1,
    name = "Original Infection"
)

# store the bar graph values
databror = [databr, dataor, databr1, dataor1]

# create the bar graph
layoutorbr = go.Layout(title = 'Severe vs Breakthrough Virus Effects', legend_title='Severe and Infection Status', xaxis_title='Continent', yaxis_title='% Percentage Protected/Susceptible')
figorbr = go.Figure(data = databror, layout = layoutorbr)

figorbr.show()

# Bar Graph of Breakthrough Susceptible and Protection against Omicron

## In this graph we see most people are more likely to be Susceptible from Breakthrough Omicron Infection compared to being protected Omicron Virus which is a huge deal compared to the last graph where the breakthrough virus was less percentage for susceptible even when vaccinated. We see that there is less percentage of protection against the Infection Omicron and an increase in Breakthrough Omicron compared to the last graph where people are less susceptible to get the breakthrough virus and infection. This shows that people getting vaccinated does have a higher chance of getting Omicron compared to the original virus. 

In [26]:
# Since they are stored in dictionary, take the values and keys of the omicron virus and infection and 
# breakthrough omicron virus and infection and use them to make 
# a bar graph to show the comparison of the people that are vaccinated how many of them would be protected and how many
# even if they are vaccinated would get the virus and the infection
country14 = list(val7.keys())
country214 = list(val8.keys())
breakthrough4 = list(val7.values())
original4 = list(val8.values())
country135 = list(val9.keys())
country2135 = list(val10.keys())
breakthrough15 = list(val9.values())
original15 = list(val10.values())
# omicron breakthrough bar graph
databr2 = go.Bar(
    x = country14,
    y = breakthrough4,
    name = "Breakthrough Omicron"
)
# omicron virus bar graph
dataor2 = go.Bar(
    x = country214,
    y = original4,
    name = "Original Omicron"
)
# omicron breakthrough infection bar graph
databr21 = go.Bar(
    x = country135,
    y = breakthrough15,
    name = "Breakthrough Infection Omicron"
)
# omicron infection bar graph
dataor22 = go.Bar(
    x = country2135,
    y = original15,
    name = "Original Infection Omicron"
)
# store the bar graph values
databror1 = [databr2, dataor2, databr21, dataor22]

# create the bar graph
layoutorbr1 = go.Layout(title = 'Severe vs Breakthrough Virus Effects', legend_title='Severe and Infection Status', xaxis_title='Continent', yaxis_title='% Percentage')
figorbr1 = go.Figure(data = databror1, layout = layoutorbr1)

figorbr1.show()

# Percentage of Prople Protected against Original and Omicron Virus and Infected based on Percentage of People Vaccinated

## We see that as more percentage of people in different countries are vaccinated, they are more likely to be protected against Original Severe Virus compared to the other infection and Omicron Virus. We do see in some continents some overlap between Severe Omicron and Infection Original Protection but in some continents we see that people are more likely to be protected against Infection Original and then we see that people are protected against Severe Omicron and less Protection against Omicron Infection. It is basically in decreasing order. But people are more likely to be protected against virus and infections if they are vaccinated 

In [27]:
# Using the continents find out the population vaccinated and % protected against original and omicron virus and infection
data = {}
for i in countries:
    # find the country in a continent along with population vaccinated, and % protected against original
    # virus and infection and omicron virus and infection
    af = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% Population vaccinated']]
    os = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% People Protected for OMICRON SEVERE']]
    os1 = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% People Protected for OMICRON INFECTION']]
    os21 = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% People Protected for ORGINAL SEVERE']]
    os22 = df_new.loc[df_new['Continent']== i, ['COUNTRY', '% People Protected for ORGINAL INFECTION']]
    data[i] = [
        # scatter plot for % protected against omicron virus for selected continent
        go.Scatter(
            x=af['% Population vaccinated'], 
            y=os['% People Protected for OMICRON SEVERE'], 
            name="People Protected against Severe Omicron", 
            mode='markers', 
            marker=dict(size=8), 
            text=af['COUNTRY']
        ),
         # scatter plot for % protected against omicron infection for selected continent
        go.Scatter(
            x=af['% Population vaccinated'], 
            y=os1['% People Protected for OMICRON INFECTION'], 
            name="People Protected against Infection Omicron", 
            mode='markers', 
            marker=dict(size=8), 
            text=af['COUNTRY']
        ),
         # scatter plot for % protected against original virus for selected continent
        go.Scatter(
            x=af['% Population vaccinated'], 
            y=os21['% People Protected for ORGINAL SEVERE'], 
            name="People Protected against Severe Original", 
            mode='markers', 
            marker=dict(size=8), 
            text=af['COUNTRY']
        ),
         # scatter plot for % protected against original infection for selected continent
        go.Scatter(
            x=af['% Population vaccinated'], 
            y=os22['% People Protected for ORGINAL INFECTION'], 
            name="People Protected against Infection Original", 
            mode='markers', 
            marker=dict(size=8), 
            text=af['COUNTRY']
        )
    ]

# create the scatter plot
layout = go.Layout(title='Percentage of People Protected Against Virus Based on Countries in Continents', legend_title='Vaccination Status', xaxis_title='% Vaccinated', yaxis_title='% Protected for Virus')

# function to receive the data containing information for the continent
def cont(countr_dropdown):
    fig24 = go.Figure(data=data[countr_dropdown], layout=layout)
    fig24.show()
# create a widget of each continent to find the information
count_dropdown = widgets.Dropdown(options=countries,
                                    description='Continents:', 
                                    layout={'width': '300px'})

# Attach the cont function to the dropdown widget
widgets.interact(cont, countr_dropdown=count_dropdown)

interactive(children=(Dropdown(description='Continents:', layout=Layout(width='300px'), options=('Asia', 'Euro…

<function __main__.cont(countr_dropdown)>

# Percentage of Prople Susceotible For Breakthrough Original and Omicron Virus and Infection based on Percentage of People Vaccinated

## We see that more people in different countries in a continent are susceptible to Breakthrough Omicron Infection even when they are vaccinated. Then we see the second higest on people being susceptible to Breakthrough Severe Omicron then we see people being susceptible to Breakthrough Infection Original and then we see breakthrough of Original Severe. This shows as even though more people are vaccinated, they can still get the virus and not everyone is protected against the virus. It shows that with Omicron rising that time, more people are likely to get breakthrough virus and infection even when they are vaccinated. 

In [28]:
# Using the continents find out the population vaccinated and % protected against breakthrough virus and infection
data2 = {}
for continent in countries:
    # find the country in a continent along with population vaccinated, and % protected against original
    # virus and infection and omicron virus and infection
    af2 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% Population vaccinated']]
    os2 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% SUCEPTIBLE for  BREAKTHROUGH OMICRON SEVERE']]
    os12 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% SUCEPTIBLE for  BREAKTHROUGH OMICRON INFECTION']]
    os212 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% SUCEPTIBLE for  BREAKTHROUGH  ORGINAL SEVERE']]
    os222 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% SUCEPTIBLE for  BREAKTHROUGH ORIGINAL  INFECTION']]
    data2[continent] = [
        # scatter plot for % susceptible against breakthrough omicron virus
        go.Scatter(
            x=af2['% Population vaccinated'], 
            y=os2['% SUCEPTIBLE for  BREAKTHROUGH OMICRON SEVERE'], 
            name="People Susceptible for Severe Omicron", 
            mode='markers', 
            marker=dict(size=8), 
            text=af2['COUNTRY']
        ),
         # scatter plot for % susceptible against breakthrough omicron infection
        go.Scatter(
            x=af2['% Population vaccinated'], 
            y=os12['% SUCEPTIBLE for  BREAKTHROUGH OMICRON INFECTION'], 
            name="People Susceptible for Infection Omicron", 
            mode='markers', 
            marker=dict(size=8), 
            text=af2['COUNTRY']
        ),
        # scatter plot for % susceptible against breakthrough original virus
        go.Scatter(
            x=af2['% Population vaccinated'], 
            y=os212['% SUCEPTIBLE for  BREAKTHROUGH  ORGINAL SEVERE'], 
            name="People Susceptible for Severe Original", 
            mode='markers', 
            marker=dict(size=8), 
            text=af2['COUNTRY']
        ),
        # scatter plot for % susceptible against original infection
        go.Scatter(
            x=af2['% Population vaccinated'], 
            y=os222['% SUCEPTIBLE for  BREAKTHROUGH ORIGINAL  INFECTION'], 
            name="People Susceptible for Infection Original", 
            mode='markers', 
            marker=dict(size=8), 
            text=af2['COUNTRY']
        )
    ]

# create the scatter plot
layout2 = go.Layout(
    title='Percentage of People Protected Against Virus Based on Countries in Continents for Omicron', 
    legend_title='Omicron Virus Status', 
    xaxis_title='% Vaccinated', 
    yaxis_title='% Susceptible for Virus'
)

# function to receive the data containing information for the continent
def display_scatter_plot2(selected_continent):
    fig = go.Figure(data=data2[selected_continent], layout=layout2)
    fig.show()

# create a widget of each continent to find the information
continent_dropdown2 = widgets.Dropdown(
    options=countries,
    description='Continents:', 
    layout={'width': '300px'}
)

# Attach the display_scatter_plot2 function to the dropdown widget
widgets.interact(display_scatter_plot2, selected_continent=continent_dropdown2)

interactive(children=(Dropdown(description='Continents:', layout=Layout(width='300px'), options=('Asia', 'Euro…

<function __main__.display_scatter_plot2(selected_continent)>

# Percentage of Prople Protected Against Original Var. and BA1.n Virus and Infection based on Percentage of People Vaccinated

## While we see that people are more lilely to be protected against Original Var. Severe, then Original Var. Infection, then BA1.n Severe and least likely to be protected against BA1.n Infection compared to others. However this time compared to other graphs we have seen, this shows that even when there are countries that have less people vaccinated and countries that have more people vaccinated, it is still the same as in people can still be protected against the virus. This shows it does not matter whether there are more people vaccinated, the protection percentage would remain the same

In [29]:
# Using the continents find out the population vaccinated and % protected against BA1.n and Original Var
data23 = {}
for continent in countries:
    # find the country in a continent along with population vaccinated, and % protected against BA1.n and Original Var
    af23 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', '% Population vaccinated']]
    os23 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', 'Protection against LATEST var BA1.n SEVERE, % average']]
    os123 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', 'Protection against LATEST var BA1.n INFECTION, % average']]
    os2123 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', 'Protection against ORIGINAL Var- SEVERE, % average']]
    os2223 = df_new.loc[df_new['Continent'] == continent, ['COUNTRY', 'Protection against ORIGINAL Var- INFECTION, % average']]
    data23[continent] = [
        # scatter plot for % protection against BA1.n Severe
        go.Scatter(
            x=af23['% Population vaccinated'], 
            y=os23['Protection against LATEST var BA1.n SEVERE, % average'], 
            name="LATEST var BA1.n SEVERE, % averag", 
            mode='markers', 
            marker=dict(size=8), 
            text=af23['COUNTRY']
        ),
        # scatter plot for % protection against BA1.n Infection
        go.Scatter(
            x=af23['% Population vaccinated'], 
            y=os123['Protection against LATEST var BA1.n INFECTION, % average'], 
            name="LATEST var BA1.n INFECTION, % average", 
            mode='markers', 
            marker=dict(size=8), 
            text=af23['COUNTRY']
        ),
        # scatter plot for % protection against Original Var. Severe
        go.Scatter(
            x=af23['% Population vaccinated'], 
            y=os2123['Protection against ORIGINAL Var- SEVERE, % average'], 
            name="ORIGINAL Var- SEVERE, % average", 
            mode='markers', 
            marker=dict(size=8), 
            text=af23['COUNTRY']
        ),
         # scatter plot for % protection against Original Var. Infection
        go.Scatter(
            x=af23['% Population vaccinated'], 
            y=os2223['Protection against ORIGINAL Var- INFECTION, % average'], 
            name="ORIGINAL Var- INFECTION, % average", 
            mode='markers', 
            marker=dict(size=8), 
            text=af23['COUNTRY']
        )
    ]

# create the scatter plot
layout23 = go.Layout(
    title='Protection against Original Var vs Latest BA1.', 
    legend_title='Virus Status', 
    xaxis_title='% Vaccinated', 
    yaxis_title='% Protected for Virus'
)

# function to receive the data containing information for the continent
def display_scatter_plot23(selected3_continent):
    fig = go.Figure(data=data23[selected3_continent], layout=layout23)
    fig.show()

# create a widget of each continent to find the information
continent_dropdown23 = widgets.Dropdown(
    options=countries,
    description='Continents:', 
    layout={'width': '300px'}
)

# Attach the display_scatter_plot23 function to the dropdown widget
widgets.interact(display_scatter_plot23, selected3_continent=continent_dropdown23)

interactive(children=(Dropdown(description='Continents:', layout=Layout(width='300px'), options=('Asia', 'Euro…

<function __main__.display_scatter_plot23(selected3_continent)>